In [1]:
!pip install -U spacy==2.3.5

In [2]:
import spacy
print(spacy.__version__)

2.3.5


In [3]:
# Had to run this to make something below work.
# This is a donwgrade of numpy to a version without a bug.
# This is a temp solution until a new version of numpy has solved the bug.
!pip install numpy==1.19.3 --user

In [4]:
def is_float(n):
    try:
        support_float_with_norwegian_format = n.replace(',','.')
        float_n = float(support_float_with_norwegian_format)
    except ValueError:
        return False
    else:
        return True
    
# test
print(is_float('test'))
print(is_float('4'))
print(is_float('4.5'))
print(is_float('4,5'))

False
True
True
True


In [5]:
def is_int(n):
    try:
        float_n = float(n)
        int_n = int(float_n)
    except ValueError:
        return False
    else:
        return float_n == int_n

# test
print(is_int('test'))
print(is_int('4'))
print(is_int('4.5'))

False
True
False


In [6]:
text = "som er kjølstrukket eller som er på et tilsvarende byggetrinn før 1. juli 2016. Skip med lengde (L) under 24 meter som er kjølstrukket 30. desember 1954 eller senere, men før 18. juli 1982."

In [7]:
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher

# Blank model.
# Does not have a POS tagger.
nlp = English()
doc = nlp(text)
matcher = Matcher(nlp.vocab)
tokens_in_doc_count = len(doc)

#
# START - spaCy patterns
#

matcher.add(
    "KEEL_LAID",
    [
        [
            {"LOWER": {"IN": ["kjølstrukk","kjølstrukket"]}}
        ]
    ])

matcher.add("DATE", None, [{'IS_DIGIT': True, 'LENGTH': 4}])

#
# END - spaCy patterns
#

result = []

for match_id, token_start, token_end in matcher(doc):

    match_id_as_string = nlp.vocab.strings[match_id]
    final_token_start = token_start
    final_token_end = token_end

    spacy_pattern_detection = doc[token_start:token_end]
    spacy_pattern_detection_as_lower_text = spacy_pattern_detection.text.lower()
    
    #
    # Expand?
    #

    if match_id_as_string == "DATE" and token_start > 0:

        # At this point, DATE is just a year string. Example: 2021

        prev_word_1_token_number = token_start - 1
        prev_word_1_token = doc[prev_word_1_token_number]
        if prev_word_1_token.text in ("januar","februar","mars","april","mai","juni","juli","august","september","oktober","november","desember"):
            final_token_start = prev_word_1_token_number # expanding

            # Expand more?
            prev_word_2_token_number = token_start - 2
            prev_word_2_token = doc[prev_word_2_token_number]
            prev_word_3_token_number = token_start - 3
            prev_word_3_token = doc[prev_word_3_token_number]

            if prev_word_2_token.text == "." and is_int(prev_word_3_token.text):
                final_token_start = prev_word_3_token_number # expanding

                #
                # convert token_span to char_span.
                # char_span is needed to display correctly withdisplacy.render().
                #
                span = doc[final_token_start:final_token_end]
                span_char_start = span[0].idx
                span_char_end = span[-1].idx + len(span[-1].text)

                # return result
                identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': match_id_as_string}
                result.append(identified_entity)

                #
                # Identify prefix or suffix
                #

                if final_token_start > 0:

                    prev_word_1_token_number = final_token_start - 1
                    prev_word_1_token = doc[prev_word_1_token_number]

                    if prev_word_1_token.text.lower() == "før":

                        # Prefix detected.

                        #
                        # convert token_span to char_span.
                        # char_span is needed to display correctly withdisplacy.render().
                        #
                        span = doc[prev_word_1_token_number:final_token_start]
                        span_char_start = span[0].idx
                        span_char_end = span[-1].idx + len(span[-1].text)

                        # return result
                        identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': "DATE_PREFIX"}
                        result.append(identified_entity)


                if ((final_token_end + 1) < tokens_in_doc_count):

                    next_word_1_token_number = final_token_end
                    next_word_1_token = doc[next_word_1_token_number]
                    next_word_2_token_number = final_token_end + 1
                    next_word_2_token = doc[next_word_2_token_number]

                    if (next_word_1_token.text.lower() == "eller" and
                        next_word_2_token.text.lower() == "senere"):

                            # Suffix detected.

                            #
                            # convert token_span to char_span.
                            # char_span is needed to display correctly withdisplacy.render().
                            #
                            span = doc[next_word_1_token_number:(next_word_1_token_number + 2)]
                            span_char_start = span[0].idx
                            span_char_end = span[-1].idx + len(span[-1].text)

                            # return result
                            identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': "DATE_SUFFIX"}
                            result.append(identified_entity)

    elif match_id_as_string == "KEEL_LAID":

        #
        # convert token_span to char_span.
        # char_span is needed to display correctly withdisplacy.render().
        #
        span = doc[final_token_start:final_token_end]
        span_char_start = span[0].idx
        span_char_end = span[-1].idx + len(span[-1].text)

        # return result
        identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': match_id_as_string}
        result.append(identified_entity)

# display result with spacy styling
display_this_result = {'text': text, 'ents': sorted(result, key=lambda x: x['start']), 'title': 0}
options = {"colors": {"KEEL_LAID": "darksalmon", "DATE": "darkkhaki", "DATE_PREFIX": "lime", "DATE_SUFFIX": "mediumvioletred"}}
from spacy import displacy
displacy.render(display_this_result, style='ent', manual=True, options=options)

# Functions run from .py file

In [8]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_KEEL_LAID_no

from spacy_matching_rule_identify_KEEL_LAID_no import identify_KEEL_LAID_in_text

importlib.reload(spacy_matching_rule_identify_KEEL_LAID_no)

################
################

identify_KEEL_LAID_in_text(text)

[{'start': 7, 'end': 19, 'label': 'KEEL_LAID'},
 {'start': 66, 'end': 78, 'label': 'DATE'},
 {'start': 62, 'end': 65, 'label': 'DATE_PREFIX'},
 {'start': 122, 'end': 134, 'label': 'KEEL_LAID'},
 {'start': 135, 'end': 152, 'label': 'DATE'},
 {'start': 153, 'end': 165, 'label': 'DATE_SUFFIX'},
 {'start': 175, 'end': 188, 'label': 'DATE'},
 {'start': 171, 'end': 174, 'label': 'DATE_PREFIX'}]